In [27]:
import tweepy
import re
values = open("../.env.production", "r").read()
twitter_bearer_token = re.findall(r"TWITTER_BEARER_TOKEN=\"(.*)\"", values)[0]
get_memes_url = re.findall(r"GET_MEMES_URL=\"(.*)\"", values)[0]
langame_api_key_id = re.findall(r"LANGAME_API_KEY_ID=\"(.*)\"", values)[0]
twitter_consumer_key = re.findall(r"TWITTER_CONSUMER_KEY=\"(.*)\"", values)[0]
twitter_consumer_secret = re.findall(r"TWITTER_CONSUMER_SECRET=\"(.*)\"", values)[0]
twitter_access_token = re.findall(r"TWITTER_ACCESS_TOKEN=\"(.*)\"", values)[0]
twitter_access_secret = re.findall(r"TWITTER_ACCESS_SECRET=\"(.*)\"", values)[0]
client = tweepy.Client(twitter_bearer_token,
    consumer_secret=twitter_consumer_secret,
    consumer_key=twitter_consumer_key,
    access_token=twitter_access_token,
    access_token_secret=twitter_access_secret,
)
# fetch popular hashtags
auth = tweepy.OAuth2BearerHandler(twitter_bearer_token)
api = tweepy.API(auth)
import geocoder
def get_trends(api, loc):
    # Object that has location's latitude and longitude.
    g = geocoder.osm(loc)
    closest_loc = api.closest_trends(g.lat, g.lng)
    trends = api.get_place_trends(closest_loc[0]["woeid"])
    return trends[0]["trends"]
loc = "USA"
trends = get_trends(api, loc)
trends

[{'name': 'Nuggets',
  'url': 'http://twitter.com/search?q=Nuggets',
  'promoted_content': None,
  'query': 'Nuggets',
  'tweet_volume': 15930},
 {'name': 'Jokic',
  'url': 'http://twitter.com/search?q=Jokic',
  'promoted_content': None,
  'query': 'Jokic',
  'tweet_volume': None},
 {'name': 'Frisch',
  'url': 'http://twitter.com/search?q=Frisch',
  'promoted_content': None,
  'query': 'Frisch',
  'tweet_volume': 92334},
 {'name': 'Pueblo',
  'url': 'http://twitter.com/search?q=Pueblo',
  'promoted_content': None,
  'query': 'Pueblo',
  'tweet_volume': 291488},
 {'name': 'Quacking',
  'url': 'http://twitter.com/search?q=Quacking',
  'promoted_content': None,
  'query': 'Quacking',
  'tweet_volume': None},
 {'name': 'Semper Fi',
  'url': 'http://twitter.com/search?q=%22Semper+Fi%22',
  'promoted_content': None,
  'query': '%22Semper+Fi%22',
  'tweet_volume': None},
 {'name': 'Marines',
  'url': 'http://twitter.com/search?q=Marines',
  'promoted_content': None,
  'query': 'Marines',
  't

In [33]:
import requests
import json
import time
def gen(topics):
    headers = {
        "Content-Type": "application/json",
    }
    data = {
        "appId": langame_api_key_id,
        "topics": topics,
        "quantity": 1,
        "translated": False,
        "fixGrammar": False,
        "parallelCompletions": 3,
    }
    tries = 0
    error = "something"
    max_tries = 3
    while error and tries < max_tries:
        response = requests.post(get_memes_url, headers=headers, data=json.dumps({"data": data}))
        response_data = response.json()
        error = response_data.get("error", None)
        tries += 1
        time.sleep(1)
    if error or "result" not in response_data:
        conversation_starter, error = None, {
            "message": response_data.get("error", {}).get("message", "Unknown error"),
            "code": response.status_code,
            "status": response_data.get("error", {}).get("status", "INTERNAL_ERROR"),
            "user_message": "Something went wrong",
        }
    conversation_starter, error = response_data.get("result", {}).get("memes", None), None
    return conversation_starter, error

In [86]:
popular_hashtags = [
    "ecology",
    "gaming",
    # "sex",
    "bitcoin",
    "technology",
    "web3",
    "ai",
    # "dating",
    # "travel",
    # "health",
]

response = client.search_recent_tweets(
    query=" OR ".join([f"#{hashtag}" for hashtag in popular_hashtags]), max_results=100,
    sort_order="relevancy",
    # include likes
    tweet_fields=["public_metrics"],
) 


In [87]:
popular_tweets = [e for e in response.data if e["public_metrics"]["like_count"] > 10]
[e.text for e in popular_tweets]

['🔍 BabyFootball #Airdrop\n\n💲Reward: Up to $4 worth of CUP + $1K worth of CUP referral pool\n\n🔴 Start the airdrop bot https://t.co/5FtF0czQZy\n\n🔘 Do the tasks on the bot &amp; submit your data\n\n🔘 Details: https://t.co/7wKwSRPda7\n\n#Airdrops #BabyFootball #Bitcoin #AirdropDetective https://t.co/B86rtRd2M2',
 'تذكير ان عملة الايثريوم بـ 2018  \n\nنزلت من 1440$ الى 81$ 👀\n\n#Ethereum #bitcoin',
 'Our #NFTCollection seeks to showcase what #AI technology perceives to be the future of each country and state in the world👀\n\nHelp the planet, one mint at a time!🤗\n\nTo find out more, visit our website⬇️\n\nhttps://t.co/bs8dSZ01Ba\n\n#NFTCollection #NFTs #Minting https://t.co/XKAavxnJTE',
 '🎁 The new #INFAM PLATFORM Contest &amp; #NFT-#Drop is ongoing! \n\nComplete 6 tasks to win a #Royal Kitty #NFT 🚀\n\nRead the details here: https://t.co/nJNQXFBrFX #Airdrops #NFTs #NFTCommumity #web3 #Crew3xyz #infam #Ambassador #kitty https://t.co/1XJj5shsyD',
 '📣3 SAATLİK KISA ÇEKİLİŞ📣    \n\n🎖175 TL🎖

In [88]:
# based on tweet hashtags, generate a conversation starter and tweet it
import re
for tweet in popular_tweets:
    hashtags = list(set(re.findall(r"#(\w+)", tweet.text)))
    # split multiple words with hyphen, example: turn ArtificialIntelligence into artificial-intelligence
    topics = [re.sub(r"([a-z])([A-Z])", r"\1-\2", hashtag).lower() for hashtag in hashtags]
    # only take first 5 topics
    topics = topics[:5]
    if not topics:
        continue
    conversation_starter, error = gen(topics)
    if error or not conversation_starter:
        print(error)
        continue
    conversation_starter_text = conversation_starter[0]["content"]
    print("~" * 100)
    print(f"Tweet text: {tweet.text}")
    print(f"Generated conversation starter: {conversation_starter_text}")
    my_tweet = f"{conversation_starter_text} {' '.join([f'#{hashtag}' for hashtag in hashtags])}"
    if len(my_tweet) > 140:
        print("Tweet is too long, skipping")
        continue
    print(f"Tweeting: {my_tweet}")
    try:
        create_tweet_response = client.create_tweet(
            text=my_tweet,
            in_reply_to_tweet_id=tweet.id,
        )
    except Exception as e:
        print(e)
    print("~" * 100)
    time.sleep(5)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Tweet text: 🔍 BabyFootball #Airdrop

💲Reward: Up to $4 worth of CUP + $1K worth of CUP referral pool

🔴 Start the airdrop bot https://t.co/5FtF0czQZy

🔘 Do the tasks on the bot &amp; submit your data

🔘 Details: https://t.co/7wKwSRPda7

#Airdrops #BabyFootball #Bitcoin #AirdropDetective https://t.co/B86rtRd2M2
Generated conversation starter: Do you think airdrops bring radical new ideas to the world of finance?
Tweeting: Do you think airdrops bring radical new ideas to the world of finance? #Airdrops #BabyFootball #Airdrop #AirdropDetective #Bitcoin
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Tweet text: تذكير ان عملة الايثريوم بـ 2018  

نزلت من 1440$ الى 81$ 👀

#Ethereum #bitcoin
Generated conversation starter: Why Ethereum's blo